# Load eneralGifs

In [ ]:
# %load generalGifs.py
import os
import imageio
import shutil


# Flow:
# call initFolders()
#     tempfolder and all contents will be automatically removed
# create and save figure images in folder tempimages/
#     save the path name into a list
#        imglist = ["tempimages/img1.png", "tempimages/img2.png"...]
# pass that list and the desired name of the gif into createGif(imglist,gifname, speed)
#     note: gif will automatically be put into gifs/ folder don't include "gifs/" in the path for gifname
#     example call: createGif(imglist,"foo.gif")
#     speed is the time in between frames in seconds (default = 0.25)

def initFolders():
    # crate temp folder
    if not os.path.isdir("tempimages"):
        os.mkdir("tempimages")
    else:
        for filename in os.listdir("tempimages"):
            os.remove("tempimages/%s" % (filename))

    if not os.path.isdir("gifs"):
        os.mkdir("gifs")


def createGif(imageList, gifname, speed=0.25):
    gifname = "gifs/%s" % (gifname)
    # create the gif
    # https://stackoverflow.com/questions/753190/programmatically-generate-video-or-animated-gif-in-python

    totalFiles = len(imageList)
    imgnum = 0
    figures = []
    for img in imageList:
        imgnum += 1
        print("[%d/%d] | processing %s \t\t\t" % (imgnum, totalFiles, img), end="\r")
        figures.append(imageio.imread(img))

    imageio.mimsave(gifname, figures, duration=speed)
    print("\ngif made at % s" % (gifname))

    # remove images and temp folder
    if os.path.isdir("tempimages"):  # should always be true here
        shutil.rmtree("tempimages")
    print("Images, temp folder removed")


# Fertility
## Read in original fertility data set and make csvs with all the variants

In [ ]:
import re
import xlrd
import csv
import time
import pandas as pd


def read_fertility(variant):
#     variant = "NO CHANGE"
    # possible variant names: "ESTIMATES", "MEDIUM VARIANT", "HIGH VARIANT", "LOW VARIANT", "CONSTANT-FERTILITY",
    # "INSTANT-REPLACEMENT", "MOMENTUM", "ZERO-MIGRATION", "CONSTANT-MORTALITY", "NO CHANGE"

    past = ["Index", "Variant", "Location", "Notes", "Country Code", "1950-1955", "1955-1960",
                  "1960-1965", "1965-1970", "1970-1975", "1975-1980","1980-1985", "1985-1990", "1990-1995",
                  "1995-2000", "2000-2005", "2005-2010", "2010-2015"]
    future = ["Index", "Variant", "Location", "Notes", "Country Code", "2015-2020", "2020-2025", "2025-2030",
              "2030-2035", "2035-2040", "2040-2045", "2045-2050", "2050-2055", "2055-2060", "2060-2065", "2065-2070", 
              "2070-2075", "2075-2080", "2080-2085", "2085-2090", "2090-2095", "2095-2100"]

    if (variant == "ESTIMATES"):
        fert_names = past
    else:
        fert_names = future

# How to make a dataframe
# https://www.geeksforgeeks.org/different-ways-to-create-pandas-dataframe/
    fileName = "data/Total_fertility.xlsx"
    fertDF = pd.read_excel(fileName, sheet_name=variant, skiprows=16, names=fert_names)

    #Read country code list csv file using pandas
    countrycodefile = 'intermediate-data/all-country-list.csv'
    cf = pd.read_csv(countrycodefile, names = ['countryname', 'al2', 'al3code','Country Code','iso','region','subr','intr','rcode','subrcode','intrcode'], skiprows = 1)
    cf.drop(['countryname','al2', 'al3code','iso','subr','intr','rcode','subrcode','intrcode'],axis=1,inplace=True)    

    # Make a dataframe called "common" that is the inner join of the countriesDF and the fertDF
    common = cf.merge(fertDF, how='inner', left_on=['Country Code'], right_on=[ 'Country Code'])
    fertDF = common.drop(columns=["Notes", "Index"])  # drop countries since same as location and notes section
    to_replace = r'[ -]'
    # Save the current dataset as a csv file with the name "Fertility_[Name of variant]" in the data folder
    csv_name = (r'./data/Fertility_%s.csv' % re.sub(to_replace, "", variant.title()))
    fertDF.to_csv(csv_name)
    print(fertDF)
    return fertDF

list = ["ESTIMATES", "MEDIUM VARIANT", "HIGH VARIANT", "LOW VARIANT", "CONSTANT-FERTILITY",
    "INSTANT-REPLACEMENT", "MOMENTUM", "ZERO-MIGRATION", "CONSTANT-MORTALITY", "NO CHANGE"]

# read_fertility("MEDIUM VARIANT")
for each in list:
    read_fertility(each)

##  graph_fert() - Using the smaller csv files, graph the variants and highlight the countries we want

In [ ]:
#visualizing population
import re
import random
import pandas as pd
import bokeh.resources
from bokeh.io import export_png
from bokeh.models import Legend
from bokeh.palettes import Set3, Greens, Blues, Reds, Purples
from bokeh.layouts import row, column
from bokeh.models import CustomJS, Slider
from bokeh.plotting import figure, output_file, show, ColumnDataSource


# Function that takes the variant we want to look at and a list of countries we want to highlight and plot
# a line graph of them
def graph_fert(variant, country_names):
# Get the csv file of all the variants and make pandas dataframes for each one. Put it in a dictionary such
# that the key is the variant name and the value is the panda dataframe associated with that key
    to_replace = r'[ -]'
    var_list = ["ESTIMATES", "MEDIUM VARIANT", "HIGH VARIANT", "LOW VARIANT", "CONSTANT-FERTILITY",
    "INSTANT-REPLACEMENT", "MOMENTUM", "ZERO-MIGRATION", "CONSTANT-MORTALITY", "NO CHANGE"]
    keys = []
    for each in var_list:
        keys.append(re.sub(to_replace, "", each.title()))

# Make a dictionary
# How to make a dictionary with just the keys
# https://www.programiz.com/python-programming/methods/dictionary/fromkeys
    dict_list = dict.fromkeys(keys)
    
    for each in keys:
        dict_list[each] = pd.read_csv("data/Fertility_%s.csv" % (each))
    
    variant_fn = re.sub(to_replace, "", variant.title())
    filename = "data/Fertility_%s.csv" % (variant_fn)

    cf = dict_list[variant_fn]
# header_list is the list of the years as in the csv file that we are interested in
    header_list = list(cf.columns.values)[5:]

# Show the country name of the line we are currently hovering over
    TOOLTIPS = [
        ("Country Name", "$name")
    ]

# the headers list is a list of the start and end year, which is of type str. The plot will not work if the list
# is made of strings, so we get the start year by getting the first four characters fromeach item from
# the header_list
    headers = []
    for each in header_list:
        headers.append(each[:4])
    headers = list(map(float,headers))
    
    
# create a new plot with a datetime axis type and the wording we want
    p = figure(plot_width=1000, plot_height=500,title="%s" % (variant.capitalize()), tooltips = TOOLTIPS)
    p.xaxis.axis_label = "Year"
    p.yaxis.axis_label = "Fertility Rate"
    output_file("styling_legend_title.html", title="fertility vs year plot")
    p.legend.title = 'Countries'
    p.legend.title_text_font_style = "bold"
    p.legend.title_text_font_size = "15pt"


# Make a dictionary with the continents and color schemes for each color so it's easily accessible
    all_countries = cf.Location
    Country_Colors = {
        "Americas" : Blues,
        "Africa" : Reds,
        "Asia" : Greens,
        "Europe" : Purples
    }
    
# How to move the legend outside the graph
# https://stackoverflow.com/questions/46730609/position-the-legend-outside-the-plot-area-with-bokeh
    legendd = []
    
    for name in all_countries:
# choose your country and get all the fertility rate values of that country for all the years
        fert = cf.loc[cf["Location"]==name][header_list].values.tolist()[0]
        fert = list(map(float,fert))

# if the country we are on is in the list of countries we want to highlight, color the line according to its
# continent and make the line width bolder
        if(name in country_names):
            continent = cf.loc[cf["Location"] == name]
            continent_colors = continent["region"].values.tolist()[0]
            if variant_fn == "Estimates":
                year_to_look_at = "1950-1955"
            else:
                year_to_look_at = "2020-2025"

            country_fert = continent[year_to_look_at].values.tolist()[0]

            color_one = round(country_fert)+1
            if color_one < 3:
                color_one = 3

            if continent_colors == "Africa":
                color_two = 2
            else:
                color_two = 1
            color = Country_Colors[continent_colors][color_one][color_two]

            c = p.line(headers, fert, color=color, alpha=1, line_width=3, name = name)
            legendd.append((name, [c]))
# If the country is not in the list of countries we are interested in, draw it with a thin width and a grey color
        else:
            color = "#ACACAC"
# make a line graph with the x-axis = header, y = pops, color of the lineis navy, and transparency is 0.5
            p.line(headers, fert, color=color, alpha=0.2, line_width=1, name = name)
    legend = Legend(items=legendd, location=(20, 0))
    p.add_layout(legend, 'right')
    
    show(p)

    
# Uncomment these next to lines to export the image to the folder we want
# USE ONLY FOR MAKING THE GIFS
#     image_location = "tempimages/Fertility_%s.png" % (variant_fn)
#     export_png(p, filename=image_location)
#     return image_location

## Plot or make a gif of the fe

In [ ]:
to_look_at = ["Niger","China", "United States of America","Indonesia", "Brazil","Pakistan", "Bangladesh","Russian Federation", "Mexico", "Japan", "Ethiopia", "Nigeria", "India"]
variant_list = ["ESTIMATES", "MEDIUM VARIANT", "HIGH VARIANT", "LOW VARIANT", "CONSTANT-FERTILITY",
    "INSTANT-REPLACEMENT", "MOMENTUM", "ZERO-MIGRATION", "CONSTANT-MORTALITY", "NO CHANGE"]

changed_vars = ["ESTIMATES", "MEDIUM VARIANT", "HIGH VARIANT", "LOW VARIANT", 
    "INSTANT-REPLACEMENT", "MOMENTUM", "ZERO-MIGRATION", "NO CHANGE"]

def make_fertility_gif():
    initFolders()
    imageList = []
    gifname = "fertility.gif"
    for each in changed_vars:
        imageList.append(graph_fert(each, to_look_at))
    createGif(imageList, gifname, 0.5)
    
# make_fertility_gif()
graph_fert("MEDIUM-VARIANT", to_look_at)
